In [ ]:
# @REMOVE-FROM-TEMPLATE
from plaster.tools.ipynb_helpers.displays import restart_kernel; restart_kernel()

In [2]:
# @IMPORT-MERGE
import numpy as np
import pandas as pd
import itertools
import cv2
from plaster.tools.log.log import error, debug
from plaster.run.job import JobResult
from plaster.run.run import RunResult
from plaster.run.sigproc_v2 import sigproc_v2_worker as worker
from plaster.run import plots
from plaster.run.sigproc_v2.synth import Synth
from plaster.tools.image.coord import WH, XY, roi_shift, clip2d
from plaster.tools.utils import data
from plaster.tools.zplots import zplots
from plaster.tools.schema import check
from plaster.tools.image import imops
from plaster.tools.zap import zap
from plaster.tools.utils import utils
from plaster.tools.calibration.calibration import Calibration
z = zplots.setup()

Loading BokehJS ...

In [3]:
# @REMOVE-FROM-TEMPLATE
from plumbum import local
job = JobResult("/erisyon/internal/jobs_folder/val_calib")
run = job.runs[0]

In [4]:
# df = run.ims_import.metadata()
# df = df[["cycle_i", "pfs_offset", "stage_z"]]
# with z(_cols=2, f_x_axis_label="Cycle"):
#     z.scat(x=df.cycle_i, y=df.pfs_offset, f_y_axis_label="PFS Offset")
#     z.scat(x=df.cycle_i, y=df.stage_z, f_y_axis_label="State Z")


# Analyze Drops

In [5]:
def peaks_lost_on_cycle(run, ch_i, cy_i, snr_thresh=5.0):
    """
    If cy_i is None then it finds remainders
    """
    n_cycles = run.sigproc_v2.n_cycles

    sig = run.sigproc_v2.signal_radmat()[:, channel_i]
    snr = run.sigproc_v2.snr()[:, channel_i]
    snr_mask = snr[:, cy_i] > snr_thresh

    relative_to_cy_0 = utils.np_safe_divide(sig, sig[:, 0][:, None])
    if cy_i is None or cy_i >= n_cycles-1:
        sig_mask = np.all(relative_to_cy_0 > 0.5, axis=1)
    else:
        sig_mask = np.all(relative_to_cy_0[:, 0:cy_i+1] > 0.7, axis=1) & np.all(relative_to_cy_0[:, cy_i+1:] < 0.1, axis=1)

    mask = snr_mask & sig_mask
    return run.sigproc_v2.peaks()[mask].reset_index(drop=True), sig[mask], snr[mask]

def draw_traces(df, sig):
    n_cycles = sig.shape[1]
    xs = np.tile(np.arange(n_cycles), (len(df), 1))
    ys = sig
    xs = np.repeat(xs, 2, axis=1)
    ys = np.repeat(sig, 2, axis=1)
    xs[:, 1::2] = xs[:, 0::2] + 1
    z.multi_line(xs=xs, ys=ys)

def fit_gaussian_peaks(run, df, ch_i, cy_i, mea=9):
    n_rows = len(df)
    fit_prms = np.full((n_rows, 7), np.nan)
    fit_stds = np.full((n_rows, 7), np.nan)
    fit_sigs = np.full((n_rows,), np.nan)
    fit_nois = np.full((n_rows,), np.nan)

    for row_i, row in enumerate(df.itertuples()):
        im = run.sigproc_v2.aln_ims[row.field_i, ch_i, cy_i]

        im = imops.crop(im, off=XY(row.aln_x, row.aln_y), dim=WH(mea, mea), center=True)
        if im.shape != (mea, mea):
            continue

        fit_prm, fit_var = imops.fit_gauss2(im)

        kernel = imops.gauss2_rho_form(*fit_prm)
        hat = np.ones(kernel.shape)  # Improve
        sig, noi = _peak_radiometry(im, kernel / kernel.sum(), hat, allow_non_unity_psf_kernel=True)

        fit_prms[row_i] = fit_prm
        fit_stds[row_i] = np.sqrt(np.array(fit_var))
        fit_sigs[row_i] = sig
        fit_nois[row_i] = noi

    return fit_prms, fit_stds, fit_sigs, fit_nois

n_cycles = run.sigproc_v2.n_cycles

def _analyze_loss_cycle(cy_i, n_samples=200):
    df, sig, snr = peaks_lost_on_cycle(run, ch_i=0, cy_i=cy_i)
    df = df.sample(n_samples)
    sig = sig[df.index]
    snr = snr[df.index]
    draw_traces(df, snr)

    fit_prms, fit_stds, fit_sigs, fit_nois = fit_gaussian_peaks(run, df, ch_i=0, cy_i=cy_i)
    z.hist(sig[:, cy_i], _bins=np.linspace(0, 20_000), _range_y=(0, 40), f_title="sig")
    z.hist(fit_sigs, _bins=np.linspace(0, 20_000), _range_y=(0, 40), f_title="fit_sig")
    z.hist(fit_prms[:, 1], _bins=np.linspace(0, 5), _range_y=(0, 40), f_title="fit_std_x")

# def _analyze_loss_cycle_foo(cy_i, n_samples=200):
#     df, sig, snr = peaks_lost_on_cycle(run, ch_i=0, cy_i=cy_i)
#     df = df.sample(n_samples)
#     sig = sig[df.index]
#     snr = snr[df.index]
#     draw_traces(df, snr)

#     fit_prms, fit_stds, fit_sigs, fit_nois = fit_gaussian_peaks(run, df, ch_i=0, cy_i=cy_i-1)
#     z.hist(sig[:, cy_i], _bins=np.linspace(0, 20_000), _range_y=(0, 40), f_title="sig")
#     z.hist(fit_sigs, _bins=np.linspace(0, 20_000), _range_y=(0, 40), f_title="fit_sig")
#     z.hist(fit_prms[:, 1], _bins=np.linspace(0, 5), _range_y=(0, 40), f_title="fit_std_x")

with z(_cols=4, _size=200):
    _analyze_loss_cycle(cy_i=4)
    _analyze_loss_cycle(cy_i=5)
#     _analyze_loss_cycle_foo(cy_i=5)
    _analyze_loss_cycle(cy_i=8)

"""
The peaks seem to narrow after cycle 4 independent of if the peak is loss at 4 or 5 or remainder
Even when I fit the peaks they get darker so it isn't just a focus effect
If it was a focus effect then they should be getting narrower after cy 4 but staying the same brightness (check in simulation)
"""

ValueError: need at least one array to concatenate

In [ ]:
# Compare all peaks that dropped after cycle 4, cycle 5, and remainders


# Compare various stats about these two groups.
# Cy 0 brightness? Nope?
debug(dropped_after_cy_4.sum())
debug(remainders.sum())
with z(_cols=3):
    cy = 0
    # Brightness at cy? Somewhat
    with z(_merge=True, f_title=f"Brightness at cy {cy}", _legend=True, _bins=np.linspace(0, 18_000, 25), _density=True):
        z.hist(sig[samp_iz][dropped_after_cy_4, cy], _label="Edmaned", color="lightgray")
        z.hist(sig[samp_iz][remainders, cy], _label="Remainders", _step=True, line_width=2)

    # SNR at cy? Somewhat
    with z(_merge=True, f_title=f"SNR at cy {cy}", _legend=True, _bins=np.linspace(0, 100, 25), _density=True):
        z.hist(snr[samp_iz][dropped_after_cy_4, cy], _label="Edmaned", color="lightgray")
        z.hist(snr[samp_iz][remainders, cy], _label="Remainders", _step=True, line_width=2)

    # Dist from center? Nope
    with z(_merge=True, f_title=f"Radius", _legend=True, _density=True):
        centered_locs = locs - [512, 512]
        radius = np.sqrt(np.sum(centered_locs**2, axis=1))
        z.hist(radius[samp_iz][dropped_after_cy_4], _label="Edmaned", color="lightgray")
        z.hist(radius[samp_iz][remainders], _label="Remainders", _step=True, line_width=2)


In [ ]:
# Scatter eigen sum of PSF vs radiomwtey

# psfs = run.sigproc_v2.psfs()
# n_fields, n_channels, n_cycles, divs_h, divs_w, dim_h, dim_w = psfs.shape 
# divs = divs_h
# assert divs == divs_w

# ratios = np.zeros_like(psfs[:, :, :, :, :, 0, 0])

# for fl, ch, cy in run.sigproc_v2.fl_ch_cy_iter():
#     for y, x, in itertools.product(range(divs), range(divs)):
#         psf = psfs[fl, ch, cy, y, x]
#         ratios[fl, ch, cy, y, x] = imops.distribution_aspect_ratio(psf)
# names = ["field_i", "channel_i", "cycle_i", "reg_y", "reg_x"]
# index = pd.MultiIndex.from_product([range(s) for s in ratios.shape], names=names)
# ratio_df = pd.DataFrame(dict(ratio=ratios.flatten()), index=index).reset_index()

# bg = np.array(run.sigproc_v2.calib["regional_bg_mean.instrument_channel[0]"])
# names = ["reg_y", "reg_x"]
# index = pd.MultiIndex.from_product([range(s) for s in bg.shape], names=names)
# bg_df = pd.DataFrame(dict(bg=bg.flatten()), index=index).reset_index()

# df = run.sigproc_v2.radmats__peaks()
# df["reg_y"] = np.floor(divs * df.aln_y / 1024).astype(int)
# df["reg_x"] = np.floor(divs * df.aln_x / 1024).astype(int)
# df = df.set_index(["field_i", "channel_i", "cycle_i", "reg_y", "reg_x"]).join(
#     ratio_df.set_index(["field_i", "channel_i", "cycle_i", "reg_y", "reg_x"])
# ).reset_index()

# df = df.set_index(["reg_y", "reg_x"]).join(
#     bg_df.set_index(["reg_y", "reg_x"])
# ).reset_index()
# df

In [ ]:
# samps = data.arg_subsample(df.signal.values, 5000)
# z.scat(x=df.signal[samps], y=df.ratio[samps], alpha=0.5, f_x_axis_label="signal", f_y_axis_label="circularity")

In [ ]:
# chcy_ims = run.ims_import.ims[0]
# chcy_ims, locs, radmat, aln_offsets, aln_scores, chcy_regional_psfs = worker.sigproc_field(chcy_ims, run.sigproc_v2.params, run.sigproc_v2.calib())

In [ ]:
# psfs = chcy_regional_psfs
# n_fields = 1
# n_channels, n_cycles, divs_h, divs_w, dim_h, dim_w = psfs.shape 
# #n_cycles = 8   # HACK

# assert divs_h == divs_w
# divs = divs_h
# assert dim_h == dim_w
# dim = dim_h
# with z(_cols=n_cycles, _size=dim*divs*3, _notools=True, _noaxes=True, _cspan=np.percentile(psfs, (0, 100))):
#     for ch, cy in itertools.product(range(n_channels), range(n_cycles)):
#         comp = np.zeros((divs * dim, divs * dim))
#         for y, x in itertools.product(range(divs), range(divs)):
#             comp[y*dim:(y+1)*dim, x*dim:(x+1)*dim] = psfs[ch, cy, y, x]
#         z.im(comp, f_title=f"ch{ch} cy{cy}")
# np.all(psfs[0, 0, 0, 0] == psfs[0, 1, 0, 0])

In [ ]:
# psf = psfs[0, 3, 0, 0]
# z.im(psf)
# imops.eigen_moments(psf)

In [ ]:
# psf = psfs[0, 5, 3, 3]
# z.im(psf)
# imops.eigen_moments(psf)

In [ ]:
# with z(_cspan=np.percentile(chcy_ims, (0, 99)), _cols=2, _size=500):
#     z.im(chcy_ims[0, 4])
#     z.im(chcy_ims[0, 5])

# Quality

In [ ]:
for ch_i in range(run.ims_import.n_channels):
    qdf = run.ims_import.qualities()
    quality = qdf[qdf.channel_i == ch_i].sort_values(["quality"])
    z.hist(quality.quality, _size_x=800, _size_y=150, f_title=f"Quality distribution channel {ch_i}")
    
    row_iz = utils.ispace(0, len(qdf), 3)
    ims = np.array([
        run.ims_import.ims[row.field_i, row.channel_i, row.cycle_i]    
        for row in qdf.iloc[row_iz].itertuples()
    ])
    
    with z(_cols=3, _cspan=np.percentile(ims, (30, 99))):
        names = ("worst", "median", "best")
        for im, name in zip(ims, names):
            z.im(im, f_title=f"Channel: {ch_i} {name}")

# Regional Illumination Balance and Channel Equalization

In [ ]:
field_i = 0
cycle_i = 0 

ims = []
for channel_i, channel_weight in zip(range(run.sigproc_v2.n_channels), run.sigproc_v2.channel_weights):
    im = run.ims_import.ims[field_i, channel_i, cycle_i]
    regional_balance = np.array(run.sigproc_v2.params.calibration["regional_illumination_balance.instrument_channel[0]"])
    balance_im = imops.interp(regional_balance, im.shape[-2:])
    ims += [np.stack((im, im * balance_im * channel_weight))]
ims = np.array(ims)
cspan = np.percentile(ims, (30, 99))

for channel_i in range(run.sigproc_v2.n_channels):
    before_im = ims[channel_i, 0]
    after_im = ims[channel_i, 1]
    with z(_cols=2, _size=500, _cspan=cspan):
        z.im(before_im, f_title=f"Before balance (field {field_i}, channel_i {channel_i}, cycle {cycle_i})")
        z.im(after_im, f_title="After balance & equalization")

In [ ]:
np.set_printoptions(precision=2)
regional_balance

# Alignment

In [ ]:
field_df = run.sigproc_v2.fields().copy()
field_df["align"] = np.sqrt(field_df.aln_x**2 + field_df.aln_y**2)
alignment = field_df.groupby("field_i").align.max().values
z.cols(alignment, f_x_axis_label="field_i", f_y_axis_label="n_pixels", f_title="Max. alignment dist.")

In [ ]:
ims = run.ims_import.ims[4, 0, :]
with z(_cols=3, _cspan=np.percentile(ims, (20, 99))):
    for cy in range(8):
        z.im(ims[cy])

In [ ]:
ims = run.sigproc_v2.aln_ims[4, 0, :]
with z(_cols=2, _cspan=np.nanpercentile(ims, (20, 99.9)), _size=700):
    z.im(ims[4])
    z.im(ims[5])

    
# Circle in the signed image all the spots that should have had zero change
# Those that are sitl present.
    
z.im_signed(  0.001 * (ims[5] - ims[4]), _size=700 )
    

# Peaks

In [ ]:
# all_snr = np.nan_to_num(run.sigproc_v2.snr(flat_chcy=True))
# all_sig = np.nan_to_num(run.sigproc_v2.signal_radmat(flat_chcy=True))
# rows_with_high_snr = np.any(all_snr > 40.0, axis=1)

# with z(
#     f_x_axis_label="SNR",
#     _size_x=800,
#     _size_y=200,
#     _bins=np.linspace(0, 90, 300),
# ):
#     good_snr = all_snr.flatten()
#     z.hist(
#         good_snr[good_snr > 1.0],
#         f_title=f"SNR distribution of all rows (excl. darks)",
#     )

#     good_snr = all_snr[~rows_with_high_snr].flatten()
#     z.hist(
#         good_snr[good_snr > 1.0],
#         f_title=f"SNR distribution of rows with no elements > 40 (excl. darks)",
#     )

#     good_snr = all_snr[rows_with_high_snr].flatten()
#     z.hist(
#         good_snr[good_snr > 1.0],
#         f_title=f"SNR distribution of rows with any element SNR > 40 (excl. darks)",
#     )

In [ ]:
field_i = 4
cycle_i = 0

im = run.sigproc_v2.aln_ims[field_i, channel_i, cycle_i]
sig = run.sigproc_v2.signal_radmat_for_field(field_i)[:, channel_i, cycle_i]
noi = run.sigproc_v2.noise_radmat_for_field(field_i)[:, channel_i, cycle_i]
snr = run.sigproc_v2.snr_for_field(field_i)[:, channel_i, cycle_i]
locs = run.sigproc_v2.locs_for_field(field_i)

keep_locs = snr > 20.0

circle_im = worker.circle_locs(im, locs[keep_locs], inner_radius=3, outer_radius=4, fill_mode="index")

with z(_merge=True, _full=True):
    z.im(im, _cper=(0, 100))

    snr_im = snr[keep_locs][circle_im.astype(int)]
    alpha_im = np.where(circle_im.astype(int) == 0, 0, 1)
    z.im_blend(snr_im, alpha_im, _palette="inferno", _nan=0)

# PSF

In [ ]:
def plot_psfs(psfs):
    n_fields, n_channels, n_cycles, divs_h, divs_w, dim_h, dim_w = psfs.shape 
    assert divs_h == divs_w
    divs = divs_h
    assert dim_h == dim_w
    dim = dim_h
    with z(_cols=n_cycles, _size=dim*divs*3, _notools=True, _noaxes=True, _cspan=np.percentile(psfs, (0, 100))):
        for fl, ch, cy in itertools.product(range(n_fields), range(n_channels), range(n_cycles)):
            comp = np.zeros((divs * dim, divs * dim))
            for y, x in itertools.product(range(divs), range(divs)):
                comp[y*dim:(y+1)*dim, x*dim:(x+1)*dim] = psfs[fl, ch, cy, y, x]
            z.im(comp, f_title=f"fl{fl} ch{ch} cy{cy}")

plot_psfs(run.sigproc_v2.psfs())


# Remainders vs Edmaned

In [ ]:
mea = 9
noise = 0.002
true_params = (1.0, 1.0, 1.0, 4, 4, 0.0, mea)
orig_im = imops.gauss2_rho_form(*true_params)
nois_im = orig_im + noise * np.random.randn(*orig_im.shape)
fit_params, fit_variance = imops.fit_gauss2(nois_im)
fit_im = imops.gauss2_rho_form(*fit_params)

debug(np.sqrt(fit_variance / np.array(fit_params)[0:6]))

with z(_cols=3, _size=200, _cspan=(0, 0.1)):
    z.im(orig_im, f_title="original")
    z.im(nois_im, f_title="with noise")
    z.im(fit_im, f_title="from fit")


In [ ]:
def im_with_circle(im, x, y):
    circle_im = worker.circle_locs(im, [(0, 0), XY(x, y)], inner_radius=5, outer_radius=10, fill_mode="index")
    clr_im = np.full_like(im, 1)
    with z(_merge=True, _full=True):
        z.im(im, _cper=(10, 99))
        alpha_im = np.where(circle_im.astype(int) == 0, 0, 1)
        z.im_blend(clr_im, alpha_im, _palette="inferno", _nan=0)    

In [ ]:
df = run.sigproc_v2.peaks().sample(100)
fit_params_cy4, fit_variances_cy4, sigs_cy4, nois_cy4 = fit_gaussian_peaks(run, df, 4)
good_fits_cy4 = np.nan_to_num(fit_variances_cy4[:, 0], nan=1e6) < 300
z.hist(fit_params_cy4[good_fits_cy4, 1], _bins=np.linspace(1, 4, 50))

fit_params_cy5, fit_variances_cy5, sigs_cy5, nois_cy5 = fit_gaussian_peaks(run, df, 5)
good_fits_cy5 = np.nan_to_num(fit_variances_cy5[:, 0], nan=1e6) < 300
z.hist(fit_params_cy5[good_fits_cy5, 1], _bins=np.linspace(1, 4, 50))

In [ ]:
with z(_merge=True, f_x_axis_label="cy4 peak width", f_y_axis_label="cy5 peak width"):
    z.scat(
        x=fit_params_cy4[good_fits_cy4, 1],
        y=fit_params_cy5[good_fits_cy4, 1],
        _range=(0, 4, 0, 4)
    )
    z.line(x=[0, 4], y=[0, 4])
    
# Yes, the cycle 5 peaks are changing shape.
# One possibility is that are narrower because the focus change
# But, does the area under the curve change?
z.im(gauss2_rho_form(*fit_params_cy4[0, :]))


# Radiometry

In [ ]:
# Histograms of derivatives
channel_i = 0
snr_thresh = 50.0

sig = run.sigproc_v2.signal_radmat()[:, channel_i]
snr = run.sigproc_v2.snr()[:, channel_i]
snr = np.max(snr, axis=1)
sig = sig[snr > snr_thresh]
deriv = np.diff(sig, axis=1, prepend=0)
span = np.max(np.abs(np.percentile(deriv, (1, 99))))

with z(_bins=np.linspace(-span, span, 100), _cols=4, _size=250, _range=(-span, span, 0, 20_000)):
    n_cycles = deriv.shape[1]
    for cy_i in range(1, n_cycles):
        z.hist(deriv[:, cy_i], f_title=f"Cycle {cy_i-1} to cycle {cy_i}", f_x_axis_label="Intensity change")

In [ ]:
# Scatter of cycle to cycle values
channel_i = 0
snr_thresh = 5.0
n_samples = 1000

sig = run.sigproc_v2.signal_radmat()[:, channel_i]
snr = run.sigproc_v2.snr()[:, channel_i]
samp_iz = data.arg_subsample(sig, n_samples)
sig = sig[samp_iz]
snr = snr[samp_iz]

snr = np.max(snr, axis=1)
sig = sig[snr > snr_thresh]

stacks = []
n_cycles = deriv.shape[1]
with z(_cols=4, _size=250, _range=(0, 60_000, 0, 60_000)):
    for cy_i in range(0, n_cycles-1):
        deltas = sig[:, cy_i:cy_i+2]
        with z(
            _merge=True,
            f_x_axis_label="int. last cycle",
            f_y_axis_label="int. this cycle",
            f_title=f"Correlation, cycle {cy_i} to {cy_i+1}",
        ):
            z.scat(
                x=deltas[:,0], y=deltas[:, 1], alpha=0.1,
            )
            z.line(x=[0, 60_000], y=[0, 60_000])
        stacks += [deltas]

all_deltas = np.vstack(stacks)
z.scat(
    x=all_deltas[:,0], y=all_deltas[:, 1], alpha=0.1,
    f_x_axis_label="int. cycle n", f_y_axis_label="int. cycle n+1", f_title=f"Correlation, all cycles"
)


In [ ]:
# Histograms by cycle
channel_i = 0


center = np.median(run.sigproc_v2.signal_radmat()[:, channel_i, 0])  #5_000  # This is a reference line, move it around as you want

with z(_cols=3, _size=250, _bins=np.linspace(0, 20000, 100), _range=(0, 20000, 0, 2000)):
    for cy in range(n_cycles):
        snr = run.sigproc_v2.snr()[:, channel_i, cy].flatten()
        sig = run.sigproc_v2.signal_radmat()[:, channel_i, cy].flatten()
        snr_mask = snr > 0
        sig = sig[snr_mask]

        with z(_merge=True):
            z.hist(sig)#[sig > 2.])
            z.line(x=[center, center], y=[0, 2000], color="red")


In [ ]:
# Histograms by region
channel_i = 0

_n_cycles = 9
snr = run.sigproc_v2.snr()[:, channel_i, :].flatten()
sig = run.sigproc_v2.signal_radmat()[:, channel_i, :].flatten()
locs = np.tile(run.sigproc_v2.locs(), (1, _n_cycles)).reshape((-1, 2))
snr_mask = snr > 20
sig = sig[snr_mask]
locs = locs[snr_mask]

top = np.max((locs[:, 0], locs[:, 1]))
divs = 5
y = utils.ispace(0, top, divs+1)
x = utils.ispace(0, top, divs+1)

center = np.median(sig[sig > 2.])

means = np.zeros((divs, divs))
with z(_cols=divs, _size=200, _bins=np.linspace(0, 30000, 100), _range=(0, 30000, 0, 1000)):
    for yi in range(len(y)-2, -1, -1):  # Reverse because imags are first quadrant but plot grids are inverted
        for xi in range(0, len(x)-1):
            mask = (y[yi] <= locs[:, 0]) & (locs[:, 0] < y[yi+1])
            mask &= (x[xi] <= locs[:, 1]) & (locs[:, 1] < x[xi+1])
            _sig = sig[mask]
            means[yi][xi] = np.mean(_sig[_sig > 2.])
            with z(_merge=True):
                z.hist(_sig[_sig > 2.])
                z.line(x=[center, center], y=[0, 1000], color="red")

#     max_mean = np.max(means)
#     for yi in range(len(y)-2, -1, -1):  # Reverse because imags are first quadrant but plot grids are inverted
#         for xi in range(0, len(x)-1):
#             mask = (y[yi] <= locs[:, 0]) & (locs[:, 0] < y[yi+1])
#             mask &= (x[xi] <= locs[:, 1]) & (locs[:, 1] < x[xi+1])
#             _sig = sig[mask]
#             mean = np.mean(_sig[_sig > 2.])
#             _sig *= max_mean / mean
#             with z(_merge=True):
#                 z.hist(_sig[_sig > 2.])
#                 z.line(x=[center, center], y=[0, 1000], color="red")


# Signal Visualization

In [ ]:
run.sigproc_v2.radmats()

In [ ]:
plots.wizard_scat_df(run, result_block="sigproc_v2")

In [ ]:
plots.wizard_xy_df(run, result_block="sigproc_v2")

In [ ]:
plots.wizard_raw_images(run, show_circles=False, peak_i_square=True, square_radius=7, result_block="sigproc_v2")